In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession.builder.appName("ComplexTypes_Lab").getOrCreate()

# Данные:
# id: int
# customer: struct (объект с именем и адресом)
# items: array (список товаров, каждый товар - массив значений [Цена, Кол-во]) - упростим для начала до списка имен
# tags: map (динамические метки)

data = [
    (1, ("Alice", "NY"), ["iPhone", "Case"], {"priority": "high", "source": "app"}),
    (2, ("Bob", "LA"), ["Samsung"], {"priority": "low"}),
    (3, ("Charlie", "TX"), [], {}) # Пустой массив и пустая карта
]

# Сложная схема
schema = StructType([
    StructField("id", IntegerType()),
    StructField("customer", StructType([
        StructField("name", StringType()),
        StructField("city", StringType())
    ])),
    StructField("items", ArrayType(StringType())),
    StructField("tags", MapType(StringType(), StringType()))
])

df = spark.createDataFrame(data, schema)
df.createOrReplaceTempView("orders")

df.printSchema()
df.show(truncate=False)

root
 |-- id: integer (nullable = true)
 |-- customer: struct (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- city: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- tags: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+---+-------------+--------------+---------------------------------+
|id |customer     |items         |tags                             |
+---+-------------+--------------+---------------------------------+
|1  |{Alice, NY}  |[iPhone, Case]|{priority -> high, source -> app}|
|2  |{Bob, LA}    |[Samsung]     |{priority -> low}                |
|3  |{Charlie, TX}|[]            |{}                               |
+---+-------------+--------------+---------------------------------+



### Доступ к элементам (Structs & Maps)

In [2]:
# Задача: Получить имя клиента (из Struct) и приоритет заказа (из Map)
sql_basic = spark.sql("""
    SELECT 
        id,
        customer.name as cust_name,       -- Dot notation для Struct
        customer.city as cust_city,
        tags['priority'] as order_prio,   -- Bracket notation для Map
        items[0] as first_item            -- Index notation для Array
    FROM orders
""")

sql_basic.show()

+---+---------+---------+----------+----------+
| id|cust_name|cust_city|order_prio|first_item|
+---+---------+---------+----------+----------+
|  1|    Alice|       NY|      high|    iPhone|
|  2|      Bob|       LA|       low|   Samsung|
|  3|  Charlie|       TX|      NULL|      NULL|
+---+---------+---------+----------+----------+



### Магия EXPLODE (Arrays)

Задача: Мы хотим посчитать, сколько раз был продан каждый товар. Сейчас товары "спрятаны" внутри массива. Нам нужно превратить их в строки.

In [3]:
# explode(items) берет массив ["iPhone", "Case"] и создает 2 строки для заказа №1.
# Остальные колонки (id, customer) дублируются.

sql_exploded = spark.sql("""
    SELECT 
        id, 
        customer.name, 
        explode(items) as item_name 
    FROM orders
""")

sql_exploded.show()
# Заметьте: Заказ №3 (Charlie) исчез! 
# Почему? Потому что массив пустой. Explode работает как Inner Join с массивом.

+---+-----+---------+
| id| name|item_name|
+---+-----+---------+
|  1|Alice|   iPhone|
|  1|Alice|     Case|
|  2|  Bob|  Samsung|
+---+-----+---------+



### POSEXPLODE и OUTER (Для профи)

Что делать, если мы хотим сохранить пустые массивы (как Left Join)? И нам нужен индекс элемента в массиве?

In [4]:
# explode_outer - сохраняет строки с пустыми массивами (будет null)
# posexplode - возвращает элемент + его позицию (индекс)

sql_advanced = spark.sql("""
    SELECT 
        id, 
        pos,
        item
    FROM orders 
    LATERAL VIEW outer posexplode(items) as pos, item
""")
# LATERAL VIEW - классический синтаксис Hive для таких операций.
# В Spark 3.x можно писать и в select, но LATERAL VIEW часто понятнее.

sql_advanced.show()
# Charlie вернулся! (pos=null, item=null)

+---+----+-------+
| id| pos|   item|
+---+----+-------+
|  1|   0| iPhone|
|  1|   1|   Case|
|  2|   0|Samsung|
|  3|NULL|   NULL|
+---+----+-------+



### Анти-паттерны и Ошибки
🚫 Ошибка 1: Data Explosion (Взрыв данных)  
- Сценарий: У вас есть массив из 1000 элементов в каждой строке. И 1 миллиард строк.
- Действие: Вы делаете explode(array).
- Результат: 1 млрд * 1000 = 1 триллион строк.
- Последствие: Shuffle умирает, диск переполняется, джоба падает.
- Совет: Всегда проверяйте size(array) перед explode или делайте это только для фильтрованных данных.

🚫 Ошибка 2: sort_array без понимания типов
- Сценарий: Сортировка массива структур.
- Проблема: Spark будет сортировать по всем полям структуры по очереди. Это может быть очень дорого.